In [44]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score
import numpy as np


# Load the data
# Load the data with ';' delimiter
measures = pd.read_csv('measures.csv', delimiter=';')
to_predict = pd.read_csv('to_predict.csv', delimiter=';')

# Convert comma decimal separator to dot and convert columns to appropriate types
measures = measures.map(lambda x: x.replace(',', '.') if isinstance(x, str) else x)
measures = measures.astype({col: float for col in measures.columns if col not in ['subject', 'activity']})
measures['subject'] = measures['subject'].astype(float)

to_predict = to_predict.map(lambda x: x.replace(',', '.') if isinstance(x, str) else x)
to_predict = to_predict.astype({col: float for col in measures.columns if col not in ['subject', 'activity']})

# Define the subjects for training and test sets
training_subjects =      [1, 3, 5, 6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26]
test_subjects = [27, 28, 29, 30]

# Split the measures dataset into training and test sets
training_set = measures[measures['subject'].isin(training_subjects)]
test_set = measures[measures['subject'].isin(test_subjects)]

# Ensure there's no overlap
assert not training_set['subject'].isin(test_subjects).any(), "Training and test sets overlap!"

# Define features and labels
X_train = training_set.drop(columns=['subject', 'activity'])
y_train = training_set['activity']

X_test = test_set.drop(columns=['subject', 'activity'])
y_test = test_set['activity']




Skaling the data

In [51]:
from sklearn.preprocessing import MinMaxScaler

# Scaling the data
scaler = MinMaxScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


1. Optimal Feature Selection:
Let's try selecting a different number of top features to see if we can find a better subset. We can use methods like Recursive Feature Elimination (RFE) with cross-validation to determine the optimal number of features.

2. Recursive Feature Elimination with Cross-Validation (RFECV):
This method helps find the best number of features by recursively removing the least important features and evaluating the model's performance.

This is for LogReg

In [ ]:
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold

# Initialize the model
#rf_model = RandomForestClassifier(n_estimators=5, random_state=42)
log_reg = LogisticRegression(max_iter=1000, C=38, dual=False)
#log_reg = DecisionTreeClassifier(max_depth=1000, min_samples_split=2)


print("start")
# Initialize RFECV
rfecv = RFECV(estimator=log_reg, step=1, cv=StratifiedKFold(5), scoring='accuracy')
print("eval")

# Fit RFECV
rfecv.fit(X_train, y_train)
print("fitted")

# Get the optimal number of features
optimal_num_features = rfecv.n_features_
print("Optimal number of features:", optimal_num_features)

# Get the support and ranking of features
selected_features = X_train.columns[rfecv.support_]
print("Selected features:", selected_features)

# Create new datasets with selected features
X_train_optimal_log_reg = X_train[selected_features]
X_test_optimal_log_reg = X_test[selected_features]

print("modelling")

# Train model using optimal features
#rf_model_optimal = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model_optimal = LogisticRegression(max_iter=10000, C=38, dual=False)
rf_model_optimal.fit(X_train_optimal_log_reg, y_train)
accuracy_optimal = rf_model_optimal.score(X_test_optimal_log_reg, y_test)
print("Test Set Accuracy with Optimal Features:", accuracy_optimal)

# Optimal number of features: 138
# Test Set Accuracy with Optimal Features: 0.9683501683501684
# with LogisticRegression(max_iter=10000, C=38, dual=False)



start
eval


This is for SVM

In [47]:
# Initialize classifiers with chosen hyperparameters
svm = SVC(C=40, gamma=0.055, kernel='rbf', probability=True)
knn = KNeighborsClassifier(n_neighbors=7, weights='distance')
dt = DecisionTreeClassifier(max_depth=1000, min_samples_split=2)
rf = RandomForestClassifier(max_depth=100, n_estimators=1000)
gb = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3)
nb = GaussianNB()
ada = AdaBoostClassifier(n_estimators=100, learning_rate=1, algorithm="SAMME")
mlp = MLPClassifier(hidden_layer_sizes=(100,), activation='relu', solver='lbfgs', alpha=0.002, learning_rate='adaptive', max_iter=2500)
xgb = XGBClassifier(n_estimators=500, learning_rate=0.2, max_depth=5, subsample=0.8, colsample_bytree=0.8, use_label_encoder=False, eval_metric='mlogloss')

In [48]:
# Define the ensemble of classifiers
ensemble = VotingClassifier(estimators=[
    #('log_reg', log_reg),
    ('svm', svm),
    #('knn', knn),
    #('dt', dt),
    #('rf', rf),
    #('gb', gb),
    #('nb', nb),
    #('ada', ada),
    #('mlp', mlp),
    #('xgb', xgb)
], voting='soft')  # Use 'soft' for probability-based voting

# Fit the ensemble model
ensemble.fit(X_train, y_train)

print("-->", ensemble.score(X_test, y_test))


# Predict on the test set
y_pred = ensemble.predict(X_test)

# Calculate accuracy and classification error
accuracy = accuracy_score(y_test, y_pred)
classification_error = 1 - accuracy

# Print the results
print("\nEnsemble Model:")
print("Test Set Accuracy:", accuracy)
# lr svm Test Set Accuracy: 0.9717171717171718
#nur svm --> 0.9676767676767677


--> 0.9703703703703703

Ensemble Model:
Test Set Accuracy: 0.9703703703703703


In [49]:
ensemble = VotingClassifier(estimators=[
    #('log_reg', log_reg),
    ('svm', svm),
    #('xgb', xgb),
    #('mlp', mlp),
], voting='soft')  # Use 'soft' for probability-based voting

ensemble.fit(X_train_optimal_log_reg, y_train)
print("top features -->", ensemble.score(X_test_optimal_log_reg, y_test))

# top features --> 0.9730639730639731 mit verbesserung der logReg features
# new highest solo SVM top features --> 0.9757575757575757

top features --> 0.9757575757575757


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import numpy as np

# Initialize classifiers
log_reg = LogisticRegression(max_iter=10000, C=38, dual=False)
svm = SVC(C=40, gamma=0.055, kernel='rbf', probability=True)

# Define different training sets for each classifier
X_train_log_reg = X_train_optimal_log_reg  # Replace with your specific dataset for Logistic Regression
X_train_svm = X_train_optimal_svm          # Replace with your specific dataset for SVM
y_train_log_reg = y_train                  # Assuming the target variable is the same

# Train each classifier on its respective dataset
log_reg.fit(X_train_log_reg, y_train_log_reg)
svm.fit(X_train_svm, y_train_log_reg)

# Make predictions on the test set
log_reg_pred_proba = log_reg.predict_proba(X_test_optimal_log_reg)  # Probability predictions for logistic regression
svm_pred_proba = svm.predict_proba(X_test_optimal_svm)          # Probability predictions for SVM

# Combine predictions using average probabilities (soft voting)
combined_pred_proba = (log_reg_pred_proba + svm_pred_proba) / 2
combined_pred = np.argmax(combined_pred_proba, axis=1)

# Evaluate the combined predictions
accuracy = accuracy_score(y_test, combined_pred)
print("Ensemble accuracy:", accuracy)
